<a href="https://colab.research.google.com/github/DeependraChaddha/Heat_Equation_Solver/blob/main/Heat_Equation_Solver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Introduction

Using Gradient Descent to solve heat equation.
The difference between LHS and RHS of Heat Equation is used as Loss function.
Will experiment with optimizers to see which works best

#Make imports

In [21]:
import torch
from torch import nn
import matplotlib
import numpy as np
import random

#Device-Agnostic Code

In [22]:
device='cuda' if torch.cuda.is_available() else 'cpu'

Set random seeds

In [23]:
def set_seed(seed=42):
    random.seed(seed)  # Python random module
    np.random.seed(seed)  # NumPy
    torch.manual_seed(seed)  # PyTorch CPU
    torch.cuda.manual_seed(seed)  # PyTorch CUDA (Single GPU)
    torch.cuda.manual_seed_all(seed)  # PyTorch CUDA (All GPUs)

    # Ensures deterministic behavior in GPU operations
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False  # Can slow down training but ensures reproducibility

# Call this at the beginning of your script
set_seed(42)

#Specifying equation and coordinate system

Will figure out how to solve for first example, will then functionalize and implement on example 2 and 3. Example 4 is time dependent so may have to use different approach for that

##Example 1
∂²T/∂x² + ∂²T/∂y² = 8π²sin(2πx)sin(2πy)

T(0,y) = 0
T(1,y) = 0
T(x,0) = 0
T(x,1) = 0


Take x=0,x=1, y=0,y=1 as edges because of the initial conditions

In [24]:
#Specify number of points in grid in each direction
#todo assert both nx,ny to be strictly greater than 1 when generalizing
nx=10
ny=10

#calculate spacing between 2 points in the grid
lx=1/(nx-1)
ly=1/(ny-1)

Make grid

In [25]:
#Make linear space of x and y and reshape to the shape(nx,1) and (ny,1) respectively (Reshaping is done so that in case of matrix multiplication or broadcasting no error is encountered)
x=torch.linspace(0,1,nx).reshape(-1,1).requires_grad_(True)
y=torch.linspace(0,1,ny).reshape(-1,1).requires_grad_(True)

#Make meshgrid
X,Y=torch.meshgrid(x.squeeze(),y.squeeze(),indexing='ij')
X.reshape(-1,1).requires_grad_(True)
Y.reshape(-1,1).requires_grad_(True)
print(X.shape)
print(Y.shape)

torch.Size([10, 10])
torch.Size([10, 10])


In [26]:
#To get (i,j) point
i=3
j=4
print(f'(i,j)th point is {X[i,j]},{Y[i,j]}')

(i,j)th point is 0.3333333432674408,0.4444444477558136


Make Neural Network to find solution of Heat Equation

In [35]:
class SolutionFunction(nn.Module):
  def __init__(self):
    super().__init__()
    self.layers=nn.Sequential(
        nn.Linear(2,10),
        nn.Tanh(),
        nn.Linear(10,10),
        nn.Tanh(),
        nn.Linear(10,10),
        nn.Tanh(),
        nn.Linear(10,1)
    )

  def forward(self,X,Y):
      #reshape before concatenation to get correct dimension
      X=X.reshape(-1,1)
      Y=Y.reshape(-1,1)
      return self.layers(torch.cat((X,Y),dim=1)).reshape(10,10)

#check if it works properly
demo_model=SolutionFunction()
###WHY IS RANDOM SEED NOT WORKING?????????###
demo_model(X,Y)

tensor([[ 5.8318e-03,  5.5934e-03,  4.9777e-03,  3.9893e-03,  2.6384e-03,
          9.4022e-04, -1.0847e-03, -3.4112e-03, -6.0101e-03, -8.8489e-03],
        [ 1.0274e-03,  7.0386e-04,  1.1593e-05, -1.0429e-03, -2.4478e-03,
         -4.1859e-03, -6.2351e-03, -8.5688e-03, -1.1157e-02, -1.3967e-02],
        [-3.9907e-03, -4.3897e-03, -5.1465e-03, -6.2529e-03, -7.6956e-03,
         -9.4557e-03, -1.1510e-02, -1.3831e-02, -1.6388e-02, -1.9149e-02],
        [-9.1793e-03, -9.6450e-03, -1.0455e-02, -1.1601e-02, -1.3067e-02,
         -1.4833e-02, -1.6875e-02, -1.9165e-02, -2.1673e-02, -2.4367e-02],
        [-1.4495e-02, -1.5020e-02, -1.5874e-02, -1.7048e-02, -1.8524e-02,
         -2.0282e-02, -2.2297e-02, -2.4541e-02, -2.6985e-02, -2.9596e-02],
        [-1.9895e-02, -2.0473e-02, -2.1364e-02, -2.2556e-02, -2.4033e-02,
         -2.5771e-02, -2.7747e-02, -2.9932e-02, -3.2298e-02, -3.4815e-02],
        [-2.5339e-02, -2.5965e-02, -2.6888e-02, -2.8092e-02, -2.9560e-02,
         -3.1270e-02, -3.3197e-0

Make Loss Function

Set Optimizer and Train


In [ ]:
#Try plotting after every few steps to visualize the training process

##Example 2
Heat Equation: ∂²T/∂x² + ∂²T/∂y² = 0

Initial Conditions:
T(0,y) = 0
T(1,y) = 0
T(x,0) = 0
T(x,1) = sin(πx)


##Example 3
Heat Equation: ∂²T/∂x² + ∂²T/∂y² = 0

Initial Conditions:
T(0,y) = 0
T(1,y) = y(1-y)
T(x,0) = 0
T(x,1) = 0


##Example 4
Heat Equation: ∂T/∂t = α(∂²T/∂x² + ∂²T/∂y²)

Initial Conditions:
T(x,y,0) = sin(πx)sin(πy)
T(0,y,t) = 0
T(1,y,t) = 0
T(x,0,t) = 0
T(x,1,t) = 0
α = 0.01
